In [1]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'g:\2025 - Ashoka\Ashoka25_Env\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
#Import the necessary libraries for training and evaluation
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, transforms
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import timm
import numpy as np
import random
import copy

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set random seed for reproducibility of the experiments
sit_number = 123456
torch.manual_seed(sit_number)
np.random.seed(sit_number)
random.seed(sit_number)

In [4]:
#1. Define transformations and augmentations for the training dataset
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0), ratio=(0.9, 1.1)), # Randomly crop with specific scale
    transforms.RandomHorizontalFlip(), # Randomly flip the image horizontally
    transforms.RandomVerticalFlip(), # Randomly flip the image vertically
    transforms.RandomRotation(20),  # Randomly rotate the image up to ±20 degrees
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2), # Random color adjustments
    transforms.RandomAffine(degrees=20, scale=(0.9, 1.1)), # Random affine transformations
    transforms.ToTensor(), # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize using ImageNet mean and standard deviation
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# 2. Path dataset dan DataLoader
train_dir = "NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/train"
val_dir = "NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/val"
test_dir = "NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test"

# Load datasets from the specified folders
train_data = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_data = datasets.ImageFolder(root=val_dir, transform=val_test_transform)
test_data = datasets.ImageFolder(root=test_dir, transform=val_test_transform)

# Calculate class weights to handle class imbalance
targets = [label for _, label in train_data] # Get all labels from the training data
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(targets), y=targets) # Compute balanced class weights
class_weights = torch.tensor(class_weights, dtype=torch.float) # Convert to tensor format

# Define DataLoader to manage data batches
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True) # Shuffle the training data
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False) # Do not shuffle validation data
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False) # Do not shuffle test data

# Output dataset counts
print(f'Train dataset size: {len(train_data)}')
print(f'Validation dataset size: {len(val_data)}')
print(f'Test dataset size: {len(test_data)}')

Train dataset size: 153
Validation dataset size: 51
Test dataset size: 53


# ConvnextV2

In [21]:
# 3. model ConvNeXt V2
model = timm.create_model('convnextv2_base', pretrained=True)

# Modify the head of the model for a binary classification task
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

In [22]:
# 4. Set up optimizer, scheduler, and device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.head.parameters(), 'lr': 0.0001},
    {'params': model.stem.parameters(), 'lr': 0.00001},
    {'params': model.stages.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

In [23]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience # How many epochs to wait before stopping
        self.min_delta = min_delta # Minimum improvement to count as progress
        self.counter = 0
        self.best_score = None
        self.early_stop = False #

    def __call__(self, val_loss, model):
        score = -val_loss # We want to minimize validation loss
        if self.best_score is None:
            self.best_score = score # Initialize best score with the first validation score
            self.save_checkpoint(model) # Save model weights
        elif score < self.best_score + self.min_delta: # If the score hasn't improved enough
            self.counter += 1 # Increment the counter
            if self.counter >= self.patience: # If the counter exceeds patience, stop
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score # Update the best score
            self.save_checkpoint(model) # Save model weights
            self.counter = 0 # Reset the counter

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict()) # Save a deep copy of the model's weights

In [24]:
# 6. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10) # Initialize early stopping

    # Initialize history lists to store metrics
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    # Loop for each epoch
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        # Training phase
        model.train() # Set model to training mode
        running_loss = 0.0 # Initialize cumulative training loss
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to device
            optimizer.zero_grad() # Zero out gradients from the previous step

            outputs = model(inputs) # Forward pass
            loss = criterion(outputs, labels) # Calculate loss
            loss.backward() # Backpropagation
            optimizer.step() # Update model parameters

            running_loss += loss.item() # Accumulate batch loss

        # Append average training loss for the epoch
        train_loss_history.append(running_loss / len(train_loader))

        # Validation phase
        val_loss = 0.0 # Initialize validation loss
        all_preds = [] # Store all predictions
        all_labels = [] # Store all true labels
        model.eval() # Set model to evaluation mode
        with torch.no_grad(): # Disable gradient calculation during evaluation
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to device
                outputs = model(inputs) # Forward pass
                loss = criterion(outputs, labels) # Calculate loss
                val_loss += loss.item() # Accumulate validation loss
                preds = torch.argmax(outputs, 1) # Get predicted class
                all_preds.extend(preds.cpu().numpy()) # Store predictions
                all_labels.extend(labels.cpu().numpy()) # Store true labels

        # Calculate metrics for the validation set
        accuracy = accuracy_score(all_labels, all_preds) # Accuracy metric
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1) # Precision metric
        recall = recall_score(all_labels, all_preds, average='binary') # Recall metric
        f1 = f1_score(all_labels, all_preds, average='binary') # F1 score metric

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel() # Get confusion matrix components
        specificity = tn / (tn + fp) # Calculate specificity

        # Print validation metrics
        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        # Append validation metrics for the epoch
        val_loss_history.append(val_loss / len(val_loader))
        val_accuracy_history.append(accuracy)

        # Update learning rate scheduler based on validation loss
        scheduler.step(val_loss / len(val_loader))
        # Check for early stopping
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/ConvnextV2_bawah.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history

In [25]:
# 7. Evaluate the Model on Test Data
def evaluate_model(model, data_loader):
    model.eval() # Set the model to evaluation mode
    all_preds, all_labels = [], [] # Initialize lists to store predictions and actual labels
    # Disable gradient calculation during evaluation to save memory and computations
    with torch.no_grad():
        # Iterate over the test data loader to get inputs and labels
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device) # Move data to the appropriate device (GPU/CPU)
            outputs = model(inputs) # Forward pass: get model outputs
            preds = torch.argmax(outputs, 1) # Get the predicted class by taking the index of the highest score
            all_preds.extend(preds.cpu().numpy()) # Store the predictions as a list of numpy arrays
            all_labels.extend(labels.cpu().numpy()) # Store the actual labels

    # Calculate performance metrics based on the true labels and predicted labels
    accuracy = accuracy_score(all_labels, all_preds) # Calculate accuracy
    precision = precision_score(all_labels, all_preds, average='binary') # Calculate precision for binary classification
    recall = recall_score(all_labels, all_preds, average='binary') # Calculate recall for binary classification
    f1 = f1_score(all_labels, all_preds, average='binary') # Calculate F1-score for binary classification

    # Confusion matrix to calculate specificity
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel() # Get confusion matrix values
    specificity = tn / (tn + fp)

    # Print all the evaluation metrics for analysis
    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

In [26]:
# Define loss function (CrossEntropyLoss) with class weights to handle class imbalance
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5) # Reduce LR when a metric stops improving

In [27]:
# 8. Train model and save the best weights
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.7176
Accuracy: 0.6863, Precision: 0.7143, Recall: 0.2632, Specificity: 0.9375, F1 Score: 0.3846
Epoch 2/100
---------------
Val Loss: 0.6582
Accuracy: 0.7059, Precision: 0.8333, Recall: 0.2632, Specificity: 0.9688, F1 Score: 0.4000
Epoch 3/100
---------------
Val Loss: 0.4690
Accuracy: 0.8039, Precision: 0.8462, Recall: 0.5789, Specificity: 0.9375, F1 Score: 0.6875
Epoch 4/100
---------------
Val Loss: 0.3631
Accuracy: 0.9020, Precision: 0.8889, Recall: 0.8421, Specificity: 0.9375, F1 Score: 0.8649
Epoch 5/100
---------------
Val Loss: 0.3266
Accuracy: 0.9020, Precision: 0.8889, Recall: 0.8421, Specificity: 0.9375, F1 Score: 0.8649
Epoch 6/100
---------------
Val Loss: 0.3204
Accuracy: 0.9020, Precision: 0.8889, Recall: 0.8421, Specificity: 0.9375, F1 Score: 0.8649
Epoch 7/100
---------------
Val Loss: 0.3182
Accuracy: 0.9216, Precision: 0.8947, Recall: 0.8947, Specificity: 0.9375, F1 Score: 0.8947
Epoch 8/100
---------------
Val Loss: 0.2975
Acc

In [28]:
# 9. Load and evaluate the best model on test data
# Initialize the model architecture without loading pretrained weights, as we will load the saved weights
best_model = timm.create_model('convnextv2_base', pretrained=False)

# Modify the classification head of the model
best_model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)), # Pool the feature maps to a single spatial point
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(best_model.num_features, 2) # Final fully connected layer with 2 output classes (binary classification)
)

# Load the saved state_dict from best_model.pth
best_model.load_state_dict(torch.load('NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/ConvnextV2_bawah.pth'))
best_model.to(device)
best_model.eval()

# Evaluate the loaded model on test data
evaluate_model(best_model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_26432\1803226123.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('NorAb_V2_Splits-

Accuracy: 0.9811, Precision: 0.9524, Recall: 1.0000, Specificity: 0.9697, F1 Score: 0.9756


In [29]:
# Importing necessary libraries for model evaluation and loading
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# Determine the device to run the model (use GPU if available, otherwise fallback to CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Initialize the Model and Load Saved Weights
# Create the ConvnextV2 model architecture without pre-trained weights
best_model = timm.create_model('convnextv2_base', pretrained=False)
best_model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(best_model.num_features, 2)
)


# Load the saved model weights from the file, mapping the weights to the appropriate device
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/ConvnextV2_bawah.pth'
best_model.load_state_dict(torch.load(model_path, map_location=device))
best_model.to(device)
best_model.eval()

# 2. Data Transformation for Test Data
# Define the transformation pipeline for test images: resize, center crop, convert to tensor, and normalize
test_transform = transforms.Compose([
    transforms.Resize(256), # Resize the image to 256x256
    transforms.CenterCrop(224), # Crop the center to 224x224
    transforms.ToTensor(), # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize the image using ImageNet statistics
])

# 3. Load Test Data
# Load the test dataset from the specified directory and apply the transformations
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval() # Set the model to evaluation mode
    all_preds, all_labels = [], [] # Initialize lists to store predictions and actual labels

    # Disable gradient calculation for evaluation to save memory and computations
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device) # Move data to the selected device
            outputs = model(inputs) # Get model predictions
            preds = torch.argmax(outputs, 1) # Get the predicted class by taking the index of the highest output score
            all_preds.extend(preds.cpu().numpy()) # Append predictions to the list
            all_labels.extend(labels.cpu().numpy()) # Append true labels to the list

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN for each class, with Abnormal first
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

    # Specificity Calculation
    specificity_abnormal = tn_normal / (tn_normal + fp_normal)
    specificity_normal = tp_abnormal / (tp_abnormal + fn_abnormal)

    print("Specificity Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Specificity: {specificity_abnormal:.4f}")
    print()

    print("Normal:")
    print(f"Specificity: {specificity_normal:.4f}")
    print()

# 5. Evaluasi Model pada Data Uji
evaluate_model(best_model, test_loader)

C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_26432\2911769911.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(model_path, map_l

Confusion Matrix:
[[32  1]
 [ 0 20]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     1.0000    0.9697    0.9846        33
      Normal     0.9524    1.0000    0.9756        20

    accuracy                         0.9811        53
   macro avg     0.9762    0.9848    0.9801        53
weighted avg     0.9820    0.9811    0.9812        53

ROC-AUC Score: 0.9848

Class: Abnormal
True Positives (TP): 32
False Positives (FP): 1
False Negatives (FN): 0
True Negatives (TN): 20

Class: Normal
True Positives (TP): 20
False Positives (FP): 0
False Negatives (FN): 1
True Negatives (TN): 32

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 1.0000
Recall: 0.9697
F1-Score: 0.9846
Support: 33.0

Normal:
Precision: 0.9524
Recall: 1.0000
F1-Score: 0.9756
Support: 20.0

Specificity Report (with 4 decimal precision):

Abnormal:
Specificity: 0.9524

Normal:
Specificity: 1.0000



# Resnet50

In [30]:
from torchvision import models

In [31]:
# 3. ResNet50 Model Initialization
model = models.resnet50(pretrained=True)

# Replacing the fully connected (fc) layer
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True


g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [32]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)


# Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.fc.parameters(), 'lr': 0.0001}, # Higher learning rate for the newly added fully connected layer
    {'params': model.conv1.parameters(), 'lr': 0.00001}, # Lower learning rate for earlier layers
    {'params': model.layer1.parameters(), 'lr': 0.00001}, # Layer 1 of ResNet
    {'params': model.layer2.parameters(), 'lr': 0.00001}, # Layer 2 of ResNet
    {'params': model.layer3.parameters(), 'lr': 0.00001}, # Layer 3 of ResNet
    {'params': model.layer4.parameters(), 'lr': 0.00001} # Layer 4 of ResNet
], weight_decay=5e-4) # Apply weight decay for regularization

In [33]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience # How many epochs to wait before stopping
        self.min_delta = min_delta # Minimum improvement to count as progress
        self.counter = 0 # Counter for non-improving epochs
        self.best_score = None # Best score seen so far
        self.early_stop = False # Whether to stop early

    def __call__(self, val_loss, model):
        score = -val_loss # We want to minimize validation loss
        if self.best_score is None:
            self.best_score = score # Initialize best score with the first validation score
            self.save_checkpoint(model) # Save model weights
        elif score < self.best_score + self.min_delta: # If the score hasn't improved enoug
            self.counter += 1 # Increment the counter
            if self.counter >= self.patience: # If the counter exceeds patience, stop
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score # Update the best score
            self.save_checkpoint(model) # Save model weights
            self.counter = 0 # Reset the counter

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict()) # Save a deep copy of the model's weights

In [34]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    # Initialize history lists
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    # Early stopping setup
    early_stopping = EarlyStopping(patience=10)

    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * inputs.size(0)

            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_loss = running_train_loss / total_train
        train_accuracy = correct_train / total_train

        # Validation
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_loss = running_val_loss / total_val
        val_accuracy = correct_val / total_val

        # Append to history
        train_loss_history.append(train_loss)
        val_loss_history.append(val_loss)
        train_accuracy_history.append(train_accuracy)
        val_accuracy_history.append(val_accuracy)

        # Print results
        print(f'Epoch {epoch+1}/{epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        # Update scheduler
        scheduler.step(val_loss)

        # Early stopping check
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    # Save the best model weights
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/ResNet50_bawah.pth')
    model.load_state_dict(early_stopping.best_model_wts)

    return model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history

In [35]:
  # 7. Evaluasi Model pada Data Uji
def evaluate_model(model, data_loader):
      model.eval() # Set the model to evaluation mode
      all_preds, all_labels = [], [] # Initialize lists to store predictions and actual labels
      # Disable gradient calculation during evaluation to save memory and computations
      with torch.no_grad():
          # Iterate over the test data loader to get inputs and labels
          for inputs, labels in data_loader:
              inputs, labels = inputs.to(device), labels.to(device) # Move data to the appropriate device (GPU/CPU)
              outputs = model(inputs) # Forward pass: get model outputs
              preds = torch.argmax(outputs, 1) # Get the predicted class by taking the index of the highest score
              all_preds.extend(preds.cpu().numpy()) # Store the predictions as a list of numpy arrays
              all_labels.extend(labels.cpu().numpy()) # Store the actual labels

      # Calculate performance metrics based on the true labels and predicted labels
      accuracy = accuracy_score(all_labels, all_preds) # Calculate accuracy
      precision = precision_score(all_labels, all_preds, average='binary') # Calculate precision for binary classification
      recall = recall_score(all_labels, all_preds, average='binary') # Calculate recall for binary classification
      f1 = f1_score(all_labels, all_preds, average='binary') # Calculate F1-score for binary classification

      # Confusion matrix to calculate specificity
      tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel() # Get confusion matrix values
      specificity = tn / (tn + fp)

      print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')
      return accuracy, precision, recall, specificity, f1

In [36]:
# Define loss function (CrossEntropyLoss) with class weights to handle class imbalance
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True) # Reduce LR when a metric stops improving

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [37]:
model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
Train Loss: 0.6859, Train Accuracy: 0.5686
Val Loss: 0.6850, Val Accuracy: 0.5294
Epoch 2/100
Train Loss: 0.6971, Train Accuracy: 0.5752
Val Loss: 0.6535, Val Accuracy: 0.5882
Epoch 3/100
Train Loss: 0.6508, Train Accuracy: 0.6601
Val Loss: 0.6533, Val Accuracy: 0.5490
Epoch 4/100
Train Loss: 0.5829, Train Accuracy: 0.7059
Val Loss: 0.6287, Val Accuracy: 0.5882
Epoch 5/100
Train Loss: 0.6144, Train Accuracy: 0.6667
Val Loss: 0.5965, Val Accuracy: 0.6863
Epoch 6/100
Train Loss: 0.5957, Train Accuracy: 0.7190
Val Loss: 0.5669, Val Accuracy: 0.6863
Epoch 7/100
Train Loss: 0.5445, Train Accuracy: 0.7712
Val Loss: 0.5401, Val Accuracy: 0.7255
Epoch 8/100
Train Loss: 0.5503, Train Accuracy: 0.7712
Val Loss: 0.5304, Val Accuracy: 0.7255
Epoch 9/100
Train Loss: 0.5140, Train Accuracy: 0.7386
Val Loss: 0.5074, Val Accuracy: 0.7255
Epoch 10/100
Train Loss: 0.5140, Train Accuracy: 0.7908
Val Loss: 0.4989, Val Accuracy: 0.7451
Epoch 11/100
Train Loss: 0.4696, Train Accuracy: 0.8562
Val

In [38]:
# Evaluation on test data
evaluate_model(model, test_loader)

Accuracy: 0.9434, Precision: 0.9474, Recall: 0.9000, Specificity: 0.9697, F1 Score: 0.9231


(0.9433962264150944,
 0.9473684210526315,
 0.9,
 0.9696969696969697,
 0.9230769230769231)

In [39]:
# Importing necessary libraries for model evaluation and loading
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

# Determine the device to run the model (use GPU if available, otherwise fallback to CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Initialize the Model and Load Saved Weights
# Create the resnet50 model architecture without pre-trained weights
model = models.resnet50(pretrained=False)

model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Load the saved model weights from the file, mapping the weights to the appropriate device
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/ResNet50_bawah.pth'
state_dict = torch.load(model_path, map_location=device, weights_only=True)

# Modify state_dict keys for compatibility with the new model structure
new_state_dict = {}
for k, v in state_dict.items():
    # Adjust the key names for the fully connected layer (fc) weights
    if k.startswith('fc.') and not (k.endswith('.1.weight') or k.endswith('.1.bias')):
        new_state_dict[k.replace('fc.', 'fc.1.')] = v
    else:
        new_state_dict[k] = v

# Load the modified state_dict into the model
model.load_state_dict(new_state_dict, strict=False)

# Move the model to the appropriate device (CPU or GPU)
model.to(device)
model.eval() # Set model to evaluation mode


# 3. Data Transformations for Testing Data
# Define the preprocessing steps for test data (resize, center crop, normalization)
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 4. Load Test Data
# Load test images using ImageFolder and apply the defined transformations
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 5. Model Evaluation Function
def evaluate_model(model, data_loader):
    model.eval() # Set the model to evaluation mode
    all_preds, all_labels = [], [] # Lists to store predictions and true labels
    with torch.no_grad(): # Disable gradient computation for efficiency
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device) # Move data to the correct device
            outputs = model(inputs) # Forward pass
            preds = torch.argmax(outputs, 1) # Get the predicted class with highest score
            all_preds.extend(preds.cpu().numpy()) # Store predictions
            all_labels.extend(labels.cpu().numpy()) # Store true labels

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN for each class, with Abnormal first
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    # Print confusion matrix details for the "Abnormal" class
    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    # Print confusion matrix details for the "Normal" class
    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

    # Specificity Calculation
    specificity_abnormal = tn_normal / (tn_normal + fp_normal)  # Specificity for Abnormal
    specificity_normal = tp_abnormal / (tp_abnormal + fn_abnormal)  # Specificity for Normal

    print("Specificity Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Specificity: {specificity_abnormal:.4f}")
    print()

    print("Normal:")
    print(f"Specificity: {specificity_normal:.4f}")
    print()

# 6. Evaluate the Model on the Test Data
evaluate_model(model, test_loader)

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Confusion Matrix:
[[32  1]
 [ 2 18]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9412    0.9697    0.9552        33
      Normal     0.9474    0.9000    0.9231        20

    accuracy                         0.9434        53
   macro avg     0.9443    0.9348    0.9392        53
weighted avg     0.9435    0.9434    0.9431        53

ROC-AUC Score: 0.9348

Class: Abnormal
True Positives (TP): 32
False Positives (FP): 1
False Negatives (FN): 2
True Negatives (TN): 18

Class: Normal
True Positives (TP): 18
False Positives (FP): 2
False Negatives (FN): 1
True Negatives (TN): 32

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9412
Recall: 0.9697
F1-Score: 0.9552
Support: 33.0

Normal:
Precision: 0.9474
Recall: 0.9000
F1-Score: 0.9231
Support: 20.0

Specificity Report (with 4 decimal precision):

Abnormal:
Specificity: 0.9474

Normal:
Specificity: 0.9412



# ResnetRS

In [40]:
# 3. Initialize ResNetRS Model
# Load the ResNetRS50 model with pre-trained weights from ImageNet
model = timm.create_model('resnetrs50', pretrained=True)

# Update the head of the model for binary classification
model.fc = nn.Sequential(
    nn.Dropout(0.5), # Dropout to prevent overfitting
    nn.Linear(model.fc.in_features, 2)  # Output layer with 2 classes
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mahasiswa\.cache\huggingface\hub\models--timm--resnetrs50.tf_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [41]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Initialize AdamW optimizer with weight decay to avoid overfitting
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)
# Use ReduceLROnPlateau scheduler to reduce the learning rate when validation loss plateaus
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
# Use CrossEntropyLoss with class weights to handle class imbalance
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [42]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [43]:
# 6. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/ResnetRS_bawah.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model

In [44]:
# 7. Model Evaluation on Test Data
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

In [45]:
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [46]:
# 8. Train and Evaluate the Model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.5921
Accuracy: 0.8235, Precision: 0.8125, Recall: 0.6842, F1 Score: 0.7429
Epoch 2/100
---------------
Val Loss: 0.5132
Accuracy: 0.8824, Precision: 0.7826, Recall: 0.9474, F1 Score: 0.8571
Epoch 3/100
---------------
Val Loss: 0.4272
Accuracy: 0.8627, Precision: 0.7500, Recall: 0.9474, F1 Score: 0.8372
Epoch 4/100
---------------
Val Loss: 0.3383
Accuracy: 0.9020, Precision: 0.8182, Recall: 0.9474, F1 Score: 0.8780
Epoch 5/100
---------------
Val Loss: 0.2759
Accuracy: 0.9216, Precision: 0.8571, Recall: 0.9474, F1 Score: 0.9000
Epoch 6/100
---------------
Val Loss: 0.2591
Accuracy: 0.9020, Precision: 0.8500, Recall: 0.8947, F1 Score: 0.8718
Epoch 7/100
---------------
Val Loss: 0.2217
Accuracy: 0.9020, Precision: 0.8182, Recall: 0.9474, F1 Score: 0.8780
Epoch 8/100
---------------
Val Loss: 0.2206
Accuracy: 0.9020, Precision: 0.8182, Recall: 0.9474, F1 Score: 0.8780
Epoch 9/100
---------------
Val Loss: 0.2263
Accuracy: 0.9020, Precision: 0.8182

In [47]:
evaluate_model(trained_model, test_loader)

Accuracy: 0.9434, Precision: 0.9048, Recall: 0.9500, F1 Score: 0.9268


In [48]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

# Specify the device to run the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 3. Model ResNetRS
model = timm.create_model('resnetrs50', pretrained=True)

model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

# 2. Load saved model weights
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/ResnetRS_bawah.pth'
state_dict = torch.load(model_path, map_location=device, weights_only=True)

new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('fc.') and not (k.endswith('.1.weight') or k.endswith('.1.bias')):
        new_state_dict[k.replace('fc.', 'fc.1.')] = v
    else:
        new_state_dict[k] = v

# Load the adjusted state_dict into the model
model.load_state_dict(new_state_dict, strict=False)

# Move model to device
model.to(device)
model.eval()


# 3. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 4. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 5. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN for each class, with Abnormal first
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

    # Specificity Calculation
    specificity_abnormal = tn_normal / (tn_normal + fp_normal)  # Specificity for Abnormal
    specificity_normal = tp_abnormal / (tp_abnormal + fn_abnormal)  # Specificity for Normal

    print("Specificity Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Specificity: {specificity_abnormal:.4f}")
    print()

    print("Normal:")
    print(f"Specificity: {specificity_normal:.4f}")
    print()

# 6. Model Evaluation on Test Data
evaluate_model(model, test_loader)

Confusion Matrix:
[[31  2]
 [ 1 19]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9688    0.9394    0.9538        33
      Normal     0.9048    0.9500    0.9268        20

    accuracy                         0.9434        53
   macro avg     0.9368    0.9447    0.9403        53
weighted avg     0.9446    0.9434    0.9437        53

ROC-AUC Score: 0.9447

Class: Abnormal
True Positives (TP): 31
False Positives (FP): 2
False Negatives (FN): 1
True Negatives (TN): 19

Class: Normal
True Positives (TP): 19
False Positives (FP): 1
False Negatives (FN): 2
True Negatives (TN): 31

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9688
Recall: 0.9394
F1-Score: 0.9538
Support: 33.0

Normal:
Precision: 0.9048
Recall: 0.9500
F1-Score: 0.9268
Support: 20.0

Specificity Report (with 4 decimal precision):

Abnormal:
Specificity: 0.9048

Normal:
Specificity: 0.9688



# DeiT

In [49]:
# Import DeiT Model from timm
model = timm.create_model('deit_base_patch16_224', pretrained=True)

# head for output to two classes
model.head = nn.Sequential(
    nn.Linear(model.num_features, 2)
)

for param in model.parameters():
    param.requires_grad = True

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mahasiswa\.cache\huggingface\hub\models--timm--deit_base_patch16_224.fb_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [50]:
# Using device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)

In [51]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [52]:
# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        # Saving the best model weights
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [53]:
# Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation phase
        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculating evaluation metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        # Update scheduler and check for early stopping
        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Load the best weight to the model
    model.load_state_dict(early_stopping.best_model_wts)

    # Save the best model to file
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/DeiT_bawah.pth')
    print("Model terbaik telah disimpan ke path yang ditentukan.")

    return model


In [54]:
# Training model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.5092
Accuracy: 0.7451, Precision: 1.0000, Recall: 0.3158, Specificity: 1.0000, F1 Score: 0.4800
Epoch 2/100
---------------
Val Loss: 0.3144
Accuracy: 0.9608, Precision: 0.9474, Recall: 0.9474, Specificity: 0.9688, F1 Score: 0.9474
Epoch 3/100
---------------
Val Loss: 0.4307
Accuracy: 0.7255, Precision: 0.5806, Recall: 0.9474, Specificity: 0.5938, F1 Score: 0.7200
Epoch 4/100
---------------
Val Loss: 0.2253
Accuracy: 0.9412, Precision: 0.9000, Recall: 0.9474, Specificity: 0.9375, F1 Score: 0.9231
Epoch 5/100
---------------
Val Loss: 0.2196
Accuracy: 0.9412, Precision: 0.9000, Recall: 0.9474, Specificity: 0.9375, F1 Score: 0.9231
Epoch 6/100
---------------
Val Loss: 0.2256
Accuracy: 0.9412, Precision: 0.9000, Recall: 0.9474, Specificity: 0.9375, F1 Score: 0.9231
Epoch 7/100
---------------
Val Loss: 0.2382
Accuracy: 0.9216, Precision: 0.8571, Recall: 0.9474, Specificity: 0.9062, F1 Score: 0.9000
Epoch 8/100
---------------
Val Loss: 0.2543
Acc

In [55]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# Specify the device to run the model
best_model = timm.create_model('deit_base_patch16_224', pretrained=True)

# head for output to two classes
best_model.head = nn.Sequential(
    nn.Linear(best_model.num_features, 2)
)

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/DeiT_bawah.pth'
best_model.load_state_dict(torch.load(model_path, map_location=device))
best_model.to(device)
best_model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

    # Specificity Calculation
    specificity_abnormal = tn_normal / (tn_normal + fp_normal)
    specificity_normal = tp_abnormal / (tp_abnormal + fn_abnormal)

    print("Specificity Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Specificity: {specificity_abnormal:.4f}")
    print()

    print("Normal:")
    print(f"Specificity: {specificity_normal:.4f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(best_model, test_loader)

C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_26432\260502957.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(model_path, map_lo

Confusion Matrix:
[[30  3]
 [ 3 17]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9091    0.9091    0.9091        33
      Normal     0.8500    0.8500    0.8500        20

    accuracy                         0.8868        53
   macro avg     0.8795    0.8795    0.8795        53
weighted avg     0.8868    0.8868    0.8868        53

ROC-AUC Score: 0.8795

Class: Abnormal
True Positives (TP): 30
False Positives (FP): 3
False Negatives (FN): 3
True Negatives (TN): 17

Class: Normal
True Positives (TP): 17
False Positives (FP): 3
False Negatives (FN): 3
True Negatives (TN): 30

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9091
Recall: 0.9091
F1-Score: 0.9091
Support: 33.0

Normal:
Precision: 0.8500
Recall: 0.8500
F1-Score: 0.8500
Support: 20.0

Specificity Report (with 4 decimal precision):

Abnormal:
Specificity: 0.8500

Normal:
Specificity: 0.9091



# MFormer

In [56]:
# 3. Model MFormer
# Using timm to load MFormer
model = timm.create_model('mixer_b16_224', pretrained=True)

# Change head for two class classification
model.head = nn.Sequential(
    nn.LayerNorm(model.num_features),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mahasiswa\.cache\huggingface\hub\models--timm--mixer_b16_224.goog_in21k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [57]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Accessing stem parameters instead of patch_embed
optimizer = optim.AdamW([
    {'params': model.head.parameters(), 'lr': 0.0001},
    {'params': model.stem.parameters(), 'lr': 0.00001},
    {'params': model.blocks.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

# Learning rate scheduler with reduction when loss stagnates
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [58]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [59]:
# 6. Training Loop with Early Stopping (tidak berubah)
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/MFormer_bawah.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [60]:
# 8. Train model and save the best weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Val Loss: 0.7816
Accuracy: 0.4314, Precision: 0.3684, Recall: 0.7368, Specificity: 0.2500, F1 Score: 0.4912
Epoch 2/100
---------------
Val Loss: 0.6731
Accuracy: 0.4706, Precision: 0.4000, Recall: 0.8421, Specificity: 0.2500, F1 Score: 0.5424
Epoch 3/100
---------------
Val Loss: 0.5857
Accuracy: 0.5686, Precision: 0.4615, Recall: 0.9474, Specificity: 0.3438, F1 Score: 0.6207
Epoch 4/100
---------------
Val Loss: 0.4977
Accuracy: 0.7255, Precision: 0.5806, Recall: 0.9474, Specificity: 0.5938, F1 Score: 0.7200
Epoch 5/100
---------------
Val Loss: 0.4572
Accuracy: 0.7059, Precision: 0.5625, Recall: 0.9474, Specificity: 0.5625, F1 Score: 0.7059
Epoch 6/100
---------------
Val Loss: 0.4390
Accuracy: 0.7255, Precision: 0.5806, Recall: 0.9474, Specificity: 0.5938, F1 Score: 0.7200
Epoch 7/100
---------------
Val Loss: 0.4131
Accuracy: 0.7255, Precision: 0.5806, Recall: 0.9474, Specificity: 0.5938, F1 Score: 0.7200
Epoch 8/100
---------------
Val Loss: 0.3798
Accuracy: 0.8235, Precision: 0.

In [61]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 1. Model MFormer
model = timm.create_model('mixer_b16_224', pretrained=True)

# Change head for two class classification
model.head = nn.Sequential(
    nn.LayerNorm(model.num_features),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/MFormer_bawah.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)

C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_26432\2135758633.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[32  1]
 [ 1 19]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9697    0.9697    0.9697        33
      Normal     0.9500    0.9500    0.9500        20

    accuracy                         0.9623        53
   macro avg     0.9598    0.9598    0.9598        53
weighted avg     0.9623    0.9623    0.9623        53

ROC-AUC Score: 0.9598

Specificity : 0.9697

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9697
Recall: 0.9697
F1-Score: 0.9697
Support: 33.0

Normal:
Precision: 0.9500
Recall: 0.9500
F1-Score: 0.9500
Support: 20.0



# Convnext

In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

In [63]:
# 1. ConvNeXt Model
model = timm.create_model('convnext_base', pretrained=True)

in_features = model.get_classifier().in_features
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Move model to device
model = model.to(device)

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mahasiswa\.cache\huggingface\hub\models--timm--convnext_base.fb_in22k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [64]:
# 2. Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.head.parameters(), 'lr': 0.0001},
    {'params': model.stem.parameters(), 'lr': 0.00001},
    {'params': model.stages.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

In [65]:
# 3. Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

In [66]:
# 4. Early Stopping Implementation
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [67]:
# 5. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50):
    early_stopping = EarlyStopping(patience=10)
    train_loss_history, val_loss_history = [], []
    train_accuracy_history, val_accuracy_history = [], []

    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        print('-' * 15)

        # Training Phase
        model.train()
        running_loss = 0.0
        all_preds, all_labels = [], []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        train_loss = running_loss / len(train_loader)
        train_accuracy = accuracy_score(all_labels, all_preds)
        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)

        # Validation Phase
        model.eval()
        val_loss = 0.0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(all_labels, all_preds)
        val_loss_history.append(val_loss)
        val_accuracy_history.append(val_accuracy)

        # Scheduler Step
        scheduler.step(val_loss)

        # Metrics Logging
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        # Early Stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    # Save the best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/Convnext_bawah.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model

In [68]:
# 7. Training the Model
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Train Loss: 0.8060, Train Accuracy: 0.6078
Val Loss: 0.4948, Val Accuracy: 0.7647
Epoch 2/100
---------------
Train Loss: 0.7160, Train Accuracy: 0.6536
Val Loss: 0.4424, Val Accuracy: 0.8235
Epoch 3/100
---------------
Train Loss: 0.6232, Train Accuracy: 0.6863
Val Loss: 0.4078, Val Accuracy: 0.8235
Epoch 4/100
---------------
Train Loss: 0.4995, Train Accuracy: 0.7582
Val Loss: 0.3750, Val Accuracy: 0.8627
Epoch 5/100
---------------
Train Loss: 0.5374, Train Accuracy: 0.7451
Val Loss: 0.3455, Val Accuracy: 0.8824
Epoch 6/100
---------------
Train Loss: 0.4713, Train Accuracy: 0.7908
Val Loss: 0.3263, Val Accuracy: 0.8627
Epoch 7/100
---------------
Train Loss: 0.3879, Train Accuracy: 0.7974
Val Loss: 0.3077, Val Accuracy: 0.9216
Epoch 8/100
---------------
Train Loss: 0.3943, Train Accuracy: 0.8170
Val Loss: 0.2932, Val Accuracy: 0.9020
Epoch 9/100
---------------
Train Loss: 0.4182, Train Accuracy: 0.8366
Val Loss: 0.2771, Val Accuracy: 0.9216
Epoch 10/100
---------------
Train Los

In [69]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model convnext
model = timm.create_model('convnext_base', pretrained=True)

in_features = model.get_classifier().in_features
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Move model to device
model = model.to(device)

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/Convnext_bawah.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_26432\3977586373.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[31  2]
 [ 1 19]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9688    0.9394    0.9538        33
      Normal     0.9048    0.9500    0.9268        20

    accuracy                         0.9434        53
   macro avg     0.9368    0.9447    0.9403        53
weighted avg     0.9446    0.9434    0.9437        53

ROC-AUC Score: 0.9447

Specificity : 0.9394

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9688
Recall: 0.9394
F1-Score: 0.9538
Support: 33.0

Normal:
Precision: 0.9048
Recall: 0.9500
F1-Score: 0.9268
Support: 20.0



# RSMamba

In [70]:
# 3. Model RSMamba
model = timm.create_model('resnet50', pretrained=True)

# Head modification for 2 class output
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning
for param in model.parameters():
    param.requires_grad = True

In [71]:
# Optimizer with different learning rates for specific layers
optimizer = optim.AdamW([
    {'params': model.fc.parameters(), 'lr': 0.0001},
    {'params': [param for name, param in model.named_parameters() if 'fc' not in name], 'lr': 0.00001}
], weight_decay=5e-4)

In [72]:
# 5. Callback Early Stopping
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [73]:
# ipython-input-10-7375c7133d09
import torch

# 6. Training loop dengan Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100, class_weights=None): # Add class_weights as an argument
    # Mendefinisikan device di dalam fungsi train_model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Check if class_weights is provided, and move it to the device if so
    if class_weights is not None:
        class_weights = class_weights.to(device)

    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/RSMamba_bawah.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model

In [74]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [75]:
# 8. Train model and save the best weight
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Val Loss: 0.6869
Accuracy: 0.5098, Precision: 0.4167, Recall: 0.7895, Specificity: 0.3438, F1 Score: 0.5455
Epoch 2/100
---------------
Val Loss: 0.6892
Accuracy: 0.5686, Precision: 0.4545, Recall: 0.7895, Specificity: 0.4375, F1 Score: 0.5769
Epoch 3/100
---------------
Val Loss: 0.6906
Accuracy: 0.5294, Precision: 0.4138, Recall: 0.6316, Specificity: 0.4688, F1 Score: 0.5000
Epoch 4/100
---------------
Val Loss: 0.6899
Accuracy: 0.6078, Precision: 0.4762, Recall: 0.5263, Specificity: 0.6562, F1 Score: 0.5000
Epoch 5/100
---------------
Val Loss: 0.6882
Accuracy: 0.6863, Precision: 0.5789, Recall: 0.5789, Specificity: 0.7500, F1 Score: 0.5789
Epoch 6/100
---------------
Val Loss: 0.6876
Accuracy: 0.7255, Precision: 0.6667, Recall: 0.5263, Specificity: 0.8438, F1 Score: 0.5882
Epoch 7/100
---------------
Val Loss: 0.6857
Accuracy: 0.6471, Precision: 0.5385, Recall: 0.3684, Specificity: 0.8125, F1 Score: 0.4375
Epoch 8/100
---------------
Val Loss: 0.6847
Accuracy: 0.6667, Precision: 0.

In [76]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model RSMamba
model = timm.create_model('resnet50', pretrained=True)

# Modifikasi head untuk output 2 kelas
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Head modification for 2 class output
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/RSMamba_bawah.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_26432\2418047716.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[31  2]
 [ 4 16]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.8857    0.9394    0.9118        33
      Normal     0.8889    0.8000    0.8421        20

    accuracy                         0.8868        53
   macro avg     0.8873    0.8697    0.8769        53
weighted avg     0.8869    0.8868    0.8855        53

ROC-AUC Score: 0.8697

Specificity : 0.9394

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.8857
Recall: 0.9394
F1-Score: 0.9118
Support: 33.0

Normal:
Precision: 0.8889
Recall: 0.8000
F1-Score: 0.8421
Support: 20.0



# Alexnet

In [77]:
# 3. Model AlexNet
from torchvision.models import alexnet

# Initialize AlexNet pretrained model
model = alexnet(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [78]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer setup
optimizer = optim.AdamW([
    {'params': model.features.parameters(), 'lr': 0.00001},
    {'params': model.classifier.parameters(), 'lr': 0.0001}
], weight_decay=5e-4)

# Scheduler setup
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [79]:
# 5. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        # Training phase
        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        train_accuracy = accuracy_score(all_labels, all_preds)
        train_loss = running_loss / len(train_loader)
        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)

        # Validation phase
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(all_labels, all_preds)
        val_loss = val_loss / len(val_loader)
        val_loss_history.append(val_loss)
        val_accuracy_history.append(val_accuracy)

        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        scheduler.step(val_loss)
        early_stopping(val_loss, model)

        if early_stopping.early_stop:
            print('Early stopping triggered.')
            break

    # Save the best model weights
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/AlexNet_bawah.pth')
    model.load_state_dict(early_stopping.best_model_wts)

    return model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history


In [80]:
# 6. Train the AlexNet model
criterion = nn.CrossEntropyLoss(weight=class_weights)
trained_model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100
)

Epoch 1/100
---------------


Train Loss: 0.6875, Train Accuracy: 0.6078
Val Loss: 0.5206, Val Accuracy: 0.6863
Epoch 2/100
---------------
Train Loss: 0.6173, Train Accuracy: 0.7386
Val Loss: 0.3904, Val Accuracy: 0.8235
Epoch 3/100
---------------
Train Loss: 0.4841, Train Accuracy: 0.7451
Val Loss: 0.3501, Val Accuracy: 0.8824
Epoch 4/100
---------------
Train Loss: 0.4165, Train Accuracy: 0.8039
Val Loss: 0.4359, Val Accuracy: 0.8039
Epoch 5/100
---------------
Train Loss: 0.3518, Train Accuracy: 0.8235
Val Loss: 0.4367, Val Accuracy: 0.8235
Epoch 6/100
---------------
Train Loss: 0.4096, Train Accuracy: 0.8039
Val Loss: 0.4451, Val Accuracy: 0.8039
Epoch 7/100
---------------
Train Loss: 0.2966, Train Accuracy: 0.8758
Val Loss: 0.3545, Val Accuracy: 0.8627
Epoch 8/100
---------------
Train Loss: 0.4286, Train Accuracy: 0.7974
Val Loss: 0.3303, Val Accuracy: 0.9020
Epoch 9/100
---------------
Train Loss: 0.3055, Train Accuracy: 0.8497
Val Loss: 0.2732, Val Accuracy: 0.9412
Epoch 10/100
---------------
Train Los

In [81]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model AlexNet
from torchvision.models import alexnet

# Initialize AlexNet pretrained model
model = alexnet(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/AlexNet_bawah.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_26432\2516213353.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[28  5]
 [ 3 17]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9032    0.8485    0.8750        33
      Normal     0.7727    0.8500    0.8095        20

    accuracy                         0.8491        53
   macro avg     0.8380    0.8492    0.8423        53
weighted avg     0.8540    0.8491    0.8503        53

ROC-AUC Score: 0.8492

Specificity : 0.8485

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9032
Recall: 0.8485
F1-Score: 0.8750
Support: 33.0

Normal:
Precision: 0.7727
Recall: 0.8500
F1-Score: 0.8095
Support: 20.0



# EfficientNet V2

In [82]:
# 3. Model EfficientNetV2
model = timm.create_model('tf_efficientnetv2_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

In [83]:
# 4. Loss function, optimizer, dan scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer dengan learning rate berbeda untuk classifier dan backbone
optimizer = optim.AdamW([
    {'params': model.classifier.parameters(), 'lr': 0.0001},
    {'params': model.conv_stem.parameters(), 'lr': 0.00001},
    {'params': model.blocks.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

In [84]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Loss function dengan class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [85]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [86]:
# 6. Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/EfficientNetV2_bawah.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [87]:
# 8. Train model and save the best weights
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Val Loss: 0.7196
Accuracy: 0.4706, Precision: 0.2143, Recall: 0.1579, Specificity: 0.6562, F1 Score: 0.1818
Epoch 2/100
---------------
Val Loss: 0.7064
Accuracy: 0.5098, Precision: 0.2857, Recall: 0.2105, Specificity: 0.6875, F1 Score: 0.2424
Epoch 3/100
---------------
Val Loss: 0.6906
Accuracy: 0.5686, Precision: 0.4000, Recall: 0.3158, Specificity: 0.7188, F1 Score: 0.3529
Epoch 4/100
---------------
Val Loss: 0.6827
Accuracy: 0.6471, Precision: 0.5333, Recall: 0.4211, Specificity: 0.7812, F1 Score: 0.4706
Epoch 5/100
---------------
Val Loss: 0.6759
Accuracy: 0.6863, Precision: 0.6000, Recall: 0.4737, Specificity: 0.8125, F1 Score: 0.5294
Epoch 6/100
---------------
Val Loss: 0.6688
Accuracy: 0.7255, Precision: 0.6923, Recall: 0.4737, Specificity: 0.8750, F1 Score: 0.5625
Epoch 7/100
---------------
Val Loss: 0.6640
Accuracy: 0.7059, Precision: 0.6429, Recall: 0.4737, Specificity: 0.8438, F1 Score: 0.5455
Epoch 8/100
---------------
Val Loss: 0.6570
Accuracy: 0.7647, Precision: 0.

In [88]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model EfficientNetV2
model = timm.create_model('tf_efficientnetv2_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/EfficientNetV2_bawah.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_26432\379371362.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locatio

Confusion Matrix:
[[31  2]
 [ 3 17]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9118    0.9394    0.9254        33
      Normal     0.8947    0.8500    0.8718        20

    accuracy                         0.9057        53
   macro avg     0.9033    0.8947    0.8986        53
weighted avg     0.9053    0.9057    0.9052        53

ROC-AUC Score: 0.8947

Specificity : 0.9394

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9118
Recall: 0.9394
F1-Score: 0.9254
Support: 33.0

Normal:
Precision: 0.8947
Recall: 0.8500
F1-Score: 0.8718
Support: 20.0



# EfficientNet BO

In [89]:
# 3. Model EfficientNet-B0
model = timm.create_model('efficientnet_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

In [90]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.blocks.parameters(), 'lr': 0.00001},
    {'params': model.classifier.parameters(), 'lr': 0.0001}
], weight_decay=5e-4)

In [91]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Loss function dengan class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [92]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [93]:
# 6. Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/EfficientNetBO_bawah.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [94]:
# 8. Train model and save the best weights
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.6556
Accuracy: 0.6275, Precision: 0.5000, Recall: 0.5789, Specificity: 0.6562, F1 Score: 0.5366
Epoch 2/100
---------------
Val Loss: 0.6322
Accuracy: 0.6078, Precision: 0.4839, Recall: 0.7895, Specificity: 0.5000, F1 Score: 0.6000
Epoch 3/100
---------------
Val Loss: 0.6228
Accuracy: 0.6471, Precision: 0.5143, Recall: 0.9474, Specificity: 0.4688, F1 Score: 0.6667
Epoch 4/100
---------------
Val Loss: 0.6196
Accuracy: 0.7059, Precision: 0.5588, Recall: 1.0000, Specificity: 0.5312, F1 Score: 0.7170
Epoch 5/100
---------------
Val Loss: 0.6185
Accuracy: 0.6863, Precision: 0.5429, Recall: 1.0000, Specificity: 0.5000, F1 Score: 0.7037
Epoch 6/100
---------------
Val Loss: 0.6176
Accuracy: 0.6863, Precision: 0.5429, Recall: 1.0000, Specificity: 0.5000, F1 Score: 0.7037
Epoch 7/100
---------------
Val Loss: 0.6129
Accuracy: 0.6863, Precision: 0.5429, Recall: 1.0000, Specificity: 0.5000, F1 Score: 0.7037
Epoch 8/100
---------------
Val Loss: 0.6086
Acc

In [95]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model EfficientNet-B0
model = timm.create_model('efficientnet_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/modelBawah/EfficientNetBO_bawah.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Bawah/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_26432\3236191635.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[29  4]
 [ 1 19]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9667    0.8788    0.9206        33
      Normal     0.8261    0.9500    0.8837        20

    accuracy                         0.9057        53
   macro avg     0.8964    0.9144    0.9022        53
weighted avg     0.9136    0.9057    0.9067        53

ROC-AUC Score: 0.9144

Specificity : 0.8788

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9667
Recall: 0.8788
F1-Score: 0.9206
Support: 33.0

Normal:
Precision: 0.8261
Recall: 0.9500
F1-Score: 0.8837
Support: 20.0

